# Install Required Packages 

In [18]:
#! pip install -U --quiet langchain_community tiktoken langchain-mistralai langchainhub chromadb langchain langgraph tavily-python

## Define Functions to build data pipeline

In [1]:

from typing import Dict, List
import json

# Define the system message and context
SYSTEM_MESSAGE = """You are an AI assistant tasked with analyzing function definition and it's dependencies to determine if sufficient information is available for test code generation."""

FUNCTION_CONTEXT = """
def add_numbers(a: int, b: int) -> int:
    return a + b
"""

In [2]:
def get_llm_response(prompt):
    '''
    NEED TO BE IMPLEMENTED.
    '''
    response = prompt
    
    return response

In [3]:
# TOBE DONE: clone repo

# TOBE DONE: get schema information w.r.t module. below cell is example.

In [4]:
# Example Index having module's schema information.
module_meta_schema = {'app.py':
                      {'abs_path':'path to file'
                      ,'function_name':{'definition':'definition goes here','description':'objective goes here..'
                                        ,'params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                        ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                        ,'caller_function':'caller function/method name goes here'
                                        ,"dependency_from_body": []
                                        ,"dependency_from_input": []
                                        }
                      ,'class_name':{'definition':'class definition goes here'
                                     ,'method_name':{'definition':'definition goes here','description':'objective goes here'
                                                     ,'params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                                     ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                                     ,'caller_function':'caller function/method name goes here'
                                                     ,"dependency_from_body": []
                                                     ,"dependency_from_input": []
                                                     }
                                     },
                      },
                      'car_emi.py':
                       {'abs_path':'path to file'
                        ,'function_name':{'definition':'definition goes here','description':'objective goes here..'
                                          ,'params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                          ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                          ,'caller_function':'caller function/method name goes here'
                                          ,"dependency_from_body": []
                                          ,"dependency_from_input": []
                                          }
                        ,'class_name':{'definition':'definition goes here'
                                       ,'method_name':{'definition':'definition goes here','description':'objective goes here'
                                                       ,'params':[{'type':'dict/list/str/int','data':'data sample if any'},]
                                                       ,'returns':{'type':'dict/list/str/int','data':'data sample if any'}
                                                       ,'caller_function':'caller function/method name goes here'
                                                       ,"dependency_from_body": []
                                                       ,"dependency_from_input": []
                                                       }
                                     },
                        },
                    }

# TOBE DONE: get in hierarchical order of dependencies for ease of retrieval.

def get_dependencies(attribute_name):
    '''
     attribute_name: function name or module name or class name
     return: text having all dependencies
    '''
    return module_meta_schema.find(attribute_name)

### Graph

In [5]:
from typing_extensions import TypedDict
from typing import List, Optional

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        function_context: given function definition as input context
        test_code: LLM generation
        additional_info: get additional information from user
        additional_info_rqd: does additional information required
        greeting: say bye when encounters max loop limit
        counter: keep track of numer of iteration 
    """
    function_context : str
    test_code : str
    additional_info_rqd : str
    additional_info : str
    greeting: Optional[str] = None
    counter : int = 0

In [10]:
from langchain.schema import Document

### Nodes Definition goes here
def generate_test_code(state):
    print("Generate test code:::::")
    function_context = state["function_context"]
    additional_info = state["additional_info"]
    
    prompt = f"{SYSTEM_MESSAGE}\n\nFunction and Dependencies:\n{function_context}\n"
    if 'additional_info' in state:
        prompt += f"\nAdditional Information:\n{additional_info}\n"
    prompt += "\nGenerate test scenarios and code for the given function:"
    
    llm_response = get_llm_response(prompt)
    state['test_code'] = llm_response

    return state

def check_test_info(state):
    print("Check whether all information given to go for generating test code::::::")
    function_context = state["function_context"]
    additional_info = state["additional_info"]
    counter = state["counter"]
    if counter is None:
        counter = 0
    
    prompt = f"{SYSTEM_MESSAGE}\n\nFunction and Dependencies:\n{function_context}\n\nAnalyze and answer the following questions (yes/no):"
    questions = [
        "Can input/output data structures be identified to create test samples?",
        "Is there sufficient clear information to create test mocks for dependencies present in body of function/method?"
    ]
    
    llm_response = get_llm_response(prompt + "\n".join(questions))
    
    # Parse LLM response into yes/no answers
    answers = [line.split(":")[-1].strip().lower() for line in llm_response.split("\n") if line]
    #response = {q: a for q, a in zip(questions, answers)}
    response = {q: 'yes' for q, a in zip(questions, answers)}
    
    state['additional_info_rqd'] = response
    state['counter'] = counter + 1
    
    return state 

def get_user_input(state):
    print("Additional information needed. Please provide::::::::")
    for question, answer in state['additional_info_rqd'].items():
        if answer == 'no':
            user_input = input(f"{question}: ")
            additional_info = "" if state.get('additional_info', "") is None else state.get('additional_info', "")
            state['additional_info_rqd'][question] = "yes"
            state['additional_info'] = additional_info + f"\n{question}: {user_input}"
    
    return state

def check_user_input(state):
    print("Decide based on user input::::::::")
    counter = state['counter']

    if counter >= 5:
        print("---DECISION: generate test code ended with counter exceed---")
        return "bye"
    else:
        print("---DECISION: check again whether all information given to go for generating test code::::::")
        return "check_test_info"

def bye(state):
    return {"greeting":"The graph has finished"}

### Edges Definition goes here
def decide_to_generate(state):
    print("Decide to generate or ask user::::::")
    
    if all([True if i.lower()=='yes' else False for i in state['additional_info_rqd'].values()]):
        print("---DECISION: all information given to go for generating test code::::::::")
        return "generate_test_code"
    else:
        print("---DECISION: take user input---")
        return "get_user_input"

### Build Graph

In [11]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("check_test_info", check_test_info)
workflow.add_node("get_user_input", get_user_input)
workflow.add_node("generate_test_code", generate_test_code)
workflow.add_node("bye", bye)

# Build graph
workflow.set_entry_point("check_test_info")
workflow.add_conditional_edges(
    "check_test_info",
    decide_to_generate,
    {
        "get_user_input": "get_user_input",
        "generate_test_code": "generate_test_code",
    },
)
workflow.add_conditional_edges(
    "get_user_input",
    check_user_input,
    {
        "check_test_info": "check_test_info",
        "bye": "bye"
    }
)
workflow.add_edge("generate_test_code", END)
workflow.add_edge('bye', END)

# Compile
app = workflow.compile()

In [12]:
# Verify Control Flow
app.channels

{'function_context': <langgraph.channels.last_value.LastValue at 0x1f8e6fd6a20>,
 'test_code': <langgraph.channels.last_value.LastValue at 0x1f8e6fd6b70>,
 'additional_info_rqd': <langgraph.channels.last_value.LastValue at 0x1f8e6fd5820>,
 'additional_info': <langgraph.channels.last_value.LastValue at 0x1f8e739fcb0>,
 'greeting': <langgraph.channels.last_value.LastValue at 0x1f8e7db7a70>,
 'counter': <langgraph.channels.last_value.LastValue at 0x1f8e7db5760>,
 '__start__': <langgraph.channels.ephemeral_value.EphemeralValue at 0x1f8e7db78f0>,
 'check_test_info': <langgraph.channels.ephemeral_value.EphemeralValue at 0x1f8e7cefb30>,
 'get_user_input': <langgraph.channels.ephemeral_value.EphemeralValue at 0x1f8e7e2f710>,
 'generate_test_code': <langgraph.channels.ephemeral_value.EphemeralValue at 0x1f8e7e2f770>,
 'bye': <langgraph.channels.ephemeral_value.EphemeralValue at 0x1f8e7e2f590>,
 'start:check_test_info': <langgraph.channels.ephemeral_value.EphemeralValue at 0x1f8e7e2dfd0>,
 'bran

### Execution

In [13]:
from pprint import pprint

inputs = {"function_context": FUNCTION_CONTEXT}

for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")

if value.get('generation',None):
    pprint(value["generation"])
else:
    print("Done.")

Check whether all information given to go for generating test code::::::
Decide to generate or ask user::::::
---DECISION: all information given to go for generating test code::::::::
'Finished running: check_test_info:'
Generate test code:::::
'Finished running: generate_test_code:'
Done.
